In [1]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

In [2]:

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution
from VRP.modify_solution import combined_rand_modification
from VRP.verify_solution import valid_path, calculate_path_distance
from VRP.compare_solutions import num_different_edges


from proj_utils.path import print_path


import pandas as pd
import simulated_annealing.solver as SA
import taboo_search.solver as TS

from multi_agent.solver_model import MultiAgentSolverModel
from multi_agent.colaboration_types import ColaborationTypes
from solver_step import SolverStep
from multi_agent.solution_pool import solution_pool_simple
%matplotlib widget

In [3]:
truck_volume_capacity = 30
truck_weight_capacity = 20000
truck_capacity = (truck_volume_capacity, truck_weight_capacity)
dists, demand, points = generator(40)

In [4]:
customer_df = pd.read_excel('data/2_detail_table_customers.xls')
deposit_df = pd.read_excel('data/4_detail_table_depots.xls')
routes = customer_df['ROUTE_ID'].unique()
dists, demand, points = format_input(routes[0],customer_df,deposit_df)

In [5]:
routes

array([2946091, 2922001, 2939484, 2958047, 2970877, 2990001, 3005971,
       2604001, 3016355, 3027038, 3044702])

In [6]:
eval_function = lambda x: calculate_path_distance(x, dists) if(valid_path(x,truck_capacity,demand)) else float('inf')

In [7]:
solver_step_SA = SA.generic_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    )

In [8]:
taboo_list_size = 10
solver_step_TS = TS.taboo_search_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    taboo_list_size
    )

In [9]:
init_state = nearest_neighbour_solution(dists, demand, truck_capacity)
init_step = SolverStep(init_state,eval_function(init_state),100)
step_function_list = [solver_step_SA, solver_step_TS]

In [14]:
solution_pool = solution_pool_simple(num_different_edges, radious=2)
sol_model = MultiAgentSolverModel(lambda : init_step, step_function_list, 2946091,20000,30,solution_pool,GA_stepSize=15,colaboration_type=ColaborationTypes.FRIENDS,QLearn_neighbor_function_list=None,QLearn_q=None,QLearn_eval_function=None)

In [15]:
for _ in range(50):
    sol_model.step()

population generated


In [16]:
sol_states = sol_model.datacollector.get_model_vars_dataframe()
print(sol_states)

                                        TheGlobalBest  TheGlobalBestValue
0   [0, 96, 100, 76, 53, 51, 31, 86, 10, 18, 14, 7...             15.2872
1   [0, 76, 100, 53, 51, 31, 78, 96, 23, 88, 39, 1...             12.7987
2   [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             11.4812
3   [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.9300
4   [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.9300
5   [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.3878
6   [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.3878
7   [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.3878
8   [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.3878
9   [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.3878
10  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.3878
11  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...             10.3878
12  [0, 76, 100, 31, 78, 96, 23, 88, 3

In [17]:
agent_state = sol_model.datacollector.get_agent_vars_dataframe()
print(agent_state)
res = agent_state.unstack()
print(res)

                 Best                                           solution
Step AgentID                                                            
1    0            inf  [0, 1, 0, 68, 0, 10, 0, 93, 85, 0, 19, 0, 24, ...
     1            inf  [0, 1, 0, 68, 0, 10, 0, 93, 85, 0, 19, 0, 24, ...
     2        15.2872  [0, 96, 100, 76, 53, 51, 31, 86, 10, 18, 14, 7...
2    0        15.2872  [0, 96, 100, 76, 53, 51, 31, 86, 10, 18, 14, 7...
     1        15.2872  [0, 96, 100, 76, 53, 51, 31, 86, 10, 18, 14, 7...
...               ...                                                ...
49   1        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...
     2        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...
50   0        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...
     1        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...
     2        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...

[150 rows x 2 columns]
            Best           